In [ ]:
# default_exp model

## Bert from the Transformers Library

In [ ]:
#export
import torch
import torch.nn as nn
import transformers

In [ ]:
#export
class HasocModel(nn.Module):
    def __init__(self, out_feat, drop=0.3, model_name='bert-base-uncased'):
        super().__init__()
        _config = transformers.BertConfig.from_pretrained(model_name)
        self.model = transformers.BertModel.from_pretrained(model_name, config=_config)
        self.tok = transformers.BertTokenizer.from_pretrained(
            pretrained_model_name_or_path=MODEL_NAME,
            do_lower_case=True,
            )
        out_size = getattr(self.model, 'pooler').dense.out_features
        self.drop = nn.Dropout(drop)
        self.classifier = nn.Linear(out_size*2, out_feat) #cus of pooling

    def forward(self, input_ids, mask=None, token_type_ids=None):
        mask = (input_ids != self.tok.pad_token_id).long()
        h_o, _ = self.model(input_ids, 
                            attention_mask=mask, 
#                             token_type_ids=token_type_ids)
                           )
        
        # pool the outputs from each timestep
        mean_ = torch.mean(h_o, 1)
        max_ = torch.max(h_o, 1)[0]
        out = torch.cat((mean_, max_), 1)
        
        out = self.drop(out)
        return self.classifier(out)